In [1]:

# coding: utf-8

# # First, let’s implement the encoder net Q(z|X), which takes input X and outputting two things: μ(X) and Σ(X), the parameters of the Gaussian.

# In[1]:


from tensorflow.examples.tutorials.mnist import input_data
from keras.layers import Input, Dense, Lambda, Conv2D, MaxPooling2D, UpSampling2D,Lambda, Flatten, Reshape, Activation
from keras.models import Model
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler
from keras.utils.generic_utils import get_custom_objects



Using TensorFlow backend.


In [2]:
# In[2]:


import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf


# In[3]:


# m : batch size
m = 50   
# n_z : latent dimension
n_z = 2
# n_epoch : number of epoches for iterations
n_epoch = 10

epsilon_std=1.0
# intermediate_dim = 512



In [3]:
#define new lrelu function
def lrelu(x, alpha=0.3):
    return tf.maximum(x, tf.multiply(x, alpha))

get_custom_objects().update({'lrelu': Activation(lrelu)})

In [4]:
# Q(z|X) -- encoder
inputs = Input(shape=(28,28,1))
x = Conv2D(16, (3, 3), activation='lrelu', padding='same')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='lrelu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='lrelu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding ='same')(x)




In [5]:
# P(X|z) -- decoder
x = Conv2D(8, (3, 3), activation='lrelu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='lrelu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='lrelu')(x)
x = UpSampling2D((2, 2))(x)
outputs = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)


In [7]:

autoencoder = Model(inputs, outputs)
encoder = Model(inputs, encoded)

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')


In [8]:
from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, y_test) = mnist.load_data()


x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), 28,28,1))
x_test = x_test.reshape((len(x_test), 28,28,1))

print (x_train.shape)
print (x_test.shape)


# In[14]:




(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [9]:
autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size= m,
                validation_data=(x_test, x_test))


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 163s - loss: 0.1722 - val_loss: 0.1284
Epoch 2/10
60000/60000 [==============================] - 160s - loss: 0.1222 - val_loss: 0.1154
Epoch 3/10
60000/60000 [==============================] - 150s - loss: 0.1133 - val_loss: 0.1093
Epoch 4/10
60000/60000 [==============================] - 149s - loss: 0.1080 - val_loss: 0.1050
Epoch 5/10
60000/60000 [==============================] - 150s - loss: 0.1046 - val_loss: 0.1047
Epoch 6/10
60000/60000 [==============================] - 155s - loss: 0.1022 - val_loss: 0.0998
Epoch 7/10
60000/60000 [==============================] - 160s - loss: 0.1002 - val_loss: 0.0977
Epoch 8/10
60000/60000 [==============================] - 162s - loss: 0.0987 - val_loss: 0.0965
Epoch 9/10
60000/60000 [==============================] - 168s - loss: 0.0974 - val_loss: 0.0954
Epoch 10/10
60000/60000 [==============================] - 32484s - loss: 0.0